In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [3]:
# Check GPU using
# tf.config.experimental.list_physical_devices()
# tf.config.experimental.get_visible_devices()

# gpu = tf.config.experimental.list_physical_devices('XLA_GPU')[0]
# tf.config.experimental.set_memory_growth(gpu, True)

# Natural Language Processing
## Live Demos

In [4]:
# model = tf.keras.models.Sequential([
#     # None in shape means it accepts any shape
#     tf.keras.layers.Input(shape = (None, 400)),
#     tf.keras.layers.LSTM(128)
# ])

In [5]:
# input_layer = tf.keras.layers.Input(shape = (None, 400))
# lstm_layer = tf.keras.layers.LSTM(128, return_state = True)(input_layer)
# model = tf.keras.models.Model(inputs = [input_layer], outputs = lstm_layer)

In [6]:
# This does not work, has to be redefined
# input_layer = tf.keras.layers.Input(shape = (None, 400))
# lstm_layer = tf.keras.layers.LSTM(128, return_state = True)(input_layer)
# regression_layer = tf.keras.layers.Dense(1)(lstm_layer)
# model = tf.keras.models.Model(inputs = [input_layer], outputs = regression_layer)

In [7]:
# lstm_layer
# [<tf.Tensor 'lstm_10/Identity:0' shape=(None, 128) dtype=float32>,
#  <tf.Tensor 'lstm_10/Identity_1:0' shape=(None, 128) dtype=float32>,
#  <tf.Tensor 'lstm_10/Identity_2:0' shape=(None, 128) dtype=float32>]

In [8]:
# Does not work!
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input(shape = (None, 400), batch_size = 8),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_state = True)),
#     tf.keras.layers.Dense(1, activation = None)
# ])

In [9]:
# model.summary()